In [ ]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import torch
import pydicom
import matplotlib.pyplot as plt

In [ ]:
DATA_DIR = CONFIG.CFG.DATA.BASE
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# number of images used to create a single 3D array of the scan
NUM_IMAGES = 8

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

In [ ]:
TRAIN_PATIENTS = train_df['Patient'].unique().tolist()
# gave the gdcm error
BAD_PATIENT_IDS = ['ID00011637202177653955184', 'ID00052637202186188008618']
ALL_TRAIN_PATIENTS = [pat for pat in TRAIN_PATIENTS if pat not in BAD_PATIENT_IDS]
ALL_TEST_PATIENTS = test_df['Patient'].unique().tolist()

In [ ]:
def get_averaged_slices(patient_id, folder_path, num_images):
    # the preprocessed array with NUM_SLICES elements
    # TODO: Handle the case when the NUM_SLICES > the actual total slices
    # TODO: resize the image to 256 X 256?

    full_path = os.path.join(folder_path, patient_id)
    # read all the dicom files for the patient into the slices list
    slices = [pydicom.read_file(os.path.join(full_path, s)) for s in os.listdir(full_path)]
    # sort the slices using their order (file number works too)
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    # final array containing averaged num_images images
    out_array = []

    # how many extra files while averaging all images into (num_images) images
    remainder_array_size = len(slices)%num_images

    # how many to average to get a single averaged image
    avging_array_size = len(slices)//num_images

    # get the first one with the remainder images
    first_array = []
    # select the first remainder + avg_arrray_size imgaes and average into one
    for slice in slices[:remainder_array_size+avging_array_size]:
        first_array.append(slice.pixel_array)
    first_avged_array = np.average(first_array, axis=0)
    out_array.append(first_avged_array)

    # after the first one get the remaining ones into out_array rolling averaging (avging_array_size) at a time.
    for i in range(remainder_array_size + avging_array_size, len(slices), avging_array_size):
        temp_array = []
        for slice in slices[i:i+avging_array_size]:
            temp_array.append(slice.pixel_array)
        avged_temp_array = np.average(temp_array, axis=0)
        out_array.append(avged_temp_array)
    
    return np.array(out_array)

In [ ]:
array_from_id = {}

In [ ]:
# store the train and test images in array_from_id
for id in ALL_TRAIN_PATIENTS:
    array_from_id[id] = get_averaged_slices(id, os.path.join(DATA_DIR, "train"), NUM_IMAGES)

for id in ALL_TEST_PATIENTS:
    array_from_id[id] = get_averaged_slices(id, os.path.join(DATA_DIR, "test"), NUM_IMAGES)

In [ ]:

out_array = []

remainder_array_size = len(slices)%NUM_SLICES
avging_array_size = len(slices)//NUM_SLICES

# get the first array with the remainder
first_array = []
first_avging_slices = slices[:remainder_array_size+avging_array_size]
for slice in first_avging_slices:
    first_array.append(slice.pixel_array)
first_avged_array = np.average(first_array, axis=0)
out_array.append(first_avged_array)
# now get the remaining averaged N-1 arrays and append to out_array so that the size is NUM_SLICES * image_size

In [ ]:
for i in range(remainder_array_size + avging_array_size, len(slices), avging_array_size):
    temp_array = []
    for slice in slices[i:i+avging_array_size]:
        temp_array.append(slice.pixel_array)
    avged_temp_array = np.average(temp_array, axis=0)
    out_array.append(avged_temp_array)

In [ ]:
out_array = np.array(out_array)
out_array.shape

In [ ]:
plt.imshow(first_avged_array)

In [ ]:
slices_array = []
for slice in slices[:9]:
    slices_array.append(slice.pixel_array)
slices_array = np.array(slices_array)

In [ ]:
averaged_array = np.average(slices_array, axis=1)

In [ ]:
averaged_array.shape

In [ ]:
plt.imshow(averaged_array)
plt.show()

In [ ]:
plt.imshow(slices[0].pixel_array)
plt.show()

In [ ]:
least = float('inf')
for id in TEST_PATIENTS:
    if len(os.listdir(os.path.join(DATA_DIR, "test", id))) < least:
        least = len(os.listdir(os.path.join(DATA_DIR, "test", id)))

print(least)

In [ ]:
m = torch.nn.Conv3d(16, 33, 3, stride=2)

In [ ]:
input = torch.randn(20, 16, 10, 50, 100)

In [ ]:
output = m(input)

In [ ]:
output.shape